In [2]:
from matplotlib import pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split,StratifiedKFold
import warnings

rcParams['figure.figsize'] = (16, 8)
pd.options.display.max_columns = 100

In [3]:
train = pd.read_csv(r"C:\Users\scott\Desktop\dankook_dacon\dataset\train.csv")
test = pd.read_csv(r"C:\Users\scott\Desktop\dankook_dacon\dataset\test.csv")
display(train.head(), test.head())

id          u          g          r          i          z  redshift  \
0   0  23.263956  20.336773  19.009500  17.672439  16.939607 -0.000081   
1   1  15.052147  14.061969  13.452418  13.268450  13.168941  0.004506   
2   2  16.786385  15.825426  15.536318  15.393535  15.350032  0.000472   
3   3  25.660638  21.188727  20.221158  19.894949  19.634649  0.000006   
4   4  24.453429  20.699170  19.042368  18.324152  17.982649 -0.000033   

    dered_u   dered_g   dered_r   dered_i   dered_z  nObserve  nDetect  \
0  23.12426  20.25779  18.95512  17.63211  16.90894        18       18   
1  14.96642  14.00452  13.41139  13.23625  13.13468         1        1   
2  16.60765  15.68659  15.44004  15.32173  15.29608         2        2   
3  25.35365  20.99465  20.08727  19.79465  19.55518         4        3   
4  23.77140  20.43384  18.86299  18.19028  17.87592        13       12   

   airmass_u  airmass_g  airmass_r  airmass_i  airmass_z  class  
0   1.189764   1.190681   1.188979   1.189355   1.190206      0  
1   1.253254   1.257836   1.248762   1.250997   1.255533      1  
2   1.022499   1.024105   1.020983   1.021730   1.023291      0  
3   1.205399   1.206058   1.204874   1.205120   1.205712      0  
4   1.193946   1.194285   1.193738   1.193826   1.194099      0

id          u          g          r          i          z  redshift  \
0  320000  19.143050  18.017200  17.473255  17.114411  16.926293  0.121303   
1  320001  18.740910  17.170473  16.369084  15.986452  15.730708 -0.000067   
2  320002  19.298048  17.880346  17.222135  16.809561  16.635239  0.110010   
3  320003  18.077467  17.156943  16.904111  16.811800  16.795520  0.000030   
4  320004  17.832467  16.764587  16.213526  15.872181  15.662084  0.068471   

    dered_u   dered_g   dered_r   dered_i   dered_z  nObserve  nDetect  \
0  19.07495  17.95495  17.43085  17.08381  16.90872         1        1   
1  15.07033  14.30976  14.39011  14.51600  14.63716         2        2   
2  19.10036  17.77263  17.14601  16.74968  16.56888         1        1   
3  17.97294  17.07353  16.84696  16.76995  16.76677         1        1   
4  17.75447  16.67943  16.15628  15.83145  15.64335         2        2   

   airmass_u  airmass_g  airmass_r  airmass_i  airmass_z  
0   1.156348   1.151487   1.161321   1.158819   1.153902  
1   1.007637   1.007383   1.007975   1.007795   1.007500  
2   1.380859   1.390490   1.371502   1.376145   1.385635  
3   1.140796   1.140808   1.140903   1.140834   1.140787  
4   1.111023   1.107658   1.114511   1.112750   1.109325

EDA

In [ ]:
train.describe()

In [ ]:
sns.heatmap(train.corr(), annot = True, annot_kws = {"size":11}, linewidths = 1)

전처리

In [ ]:
# 동시에 전처리하기 위해 training data와 test data를 concatenate

df = pd.concat([train,test])
df.head()

In [ ]:
# Machine Learning Applied to Star–Galaxy–QSO Classification and Stellar Effective Temperature Regression 논문 참조 파생변수 생성

df["u-g"] = df["u"]-df["g"]
df["g-r"] = df["g"]-df["r"]
df["r-i"] = df["r"]-df["i"]
df["i-z"] = df["i"]-df["z"]

df["d1diff"] = df["dered_u"]-df["dered_g"]
df["d2diff"] = df["dered_g"]-df["dered_r"]
df["d3diff"] = df["dered_r"]-df["dered_i"]
df["d4diff"] = df["dered_i"]-df["dered_z"]

df["minusOD"] = df["nObserve"] - df["nDetect"]
df["plusOD"] = df["nObserve"] + df["nDetect"]

In [ ]:
# EDA 이후 불필요 칼럼, 상관관계, 다중공선성 등...을 고려하여 일부 피쳐 제거

df2 = df.drop(["id","class","airmass_u","airmass_g","airmass_r","airmass_i","airmass_z"], axis=1)
df2.head()

In [ ]:
# df를 train과 test의 크기만큼 다시 분리

train2 = df2[:len(train)]
test2 = df2[len(train):]

RandomForest모델 학습

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(train, train["class"], test_size = 0.2, random_state = 42)

clf = RandomForestClassifier(n_estimators=400,
                             n_jobs=-1,
                             verbose = 1)

clf.fit(train2, train["class"])
rf_result = clf.predict_proba(test2)


Xgboost모델 학습

In [ ]:
# StratifiedKFold를 사용하여 교차검증

sfk = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
xgb_result = 0

for i,j in sfk.split(train2,train["class"]):
    x_train = train2.iloc[i]
    x_valid = train2.iloc[j]
    y_train = train["class"].iloc[i]
    y_valid = train["class"].iloc[j]

    xgb = XGBClassifier(objective='multiclass',
                         n_estimators = 500, 
                         max_depth = 9,
                         subsample = 1,
                         colsample_bytree = 1,
                         nthread= -1, 
                         )
    xgb.fit(x_train, y_train, eval_set = [(x_valid, y_valid)], early_stopping_rounds = 70)
    xgb_result += xgb.predict_proba(test2)/10


일정 비율 앙상블

In [ ]:
result = rf_result*0.4 + xgb_result*0.6
final = pd.DataFrame(result)

In [ ]:
sub = pd.read_csv(r"C:\Users\scott\Desktop\dankook_dacon\dataset\sample_submission.csv")
sub["class"] = final.argmax(1)
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)